In [2]:

%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use this for new GPUs like Ampere, Hopper GPUs (RTX 30xx, RTX 40xx, A100, H100, L40)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use this for older GPUs (V100, Tesla T4, RTX 20xx)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
# model_name = "unsloth/mistral-7b-v0.3"
model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/hice1/mmahendran7/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2024.11.10: Fast Mistral patching. Transformers:4.46.3.
   \\   /|    GPU: Tesla V100-PCIE-16GB. Max memory: 15.766 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.11.10 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:

# Now the dataset
import pandas as pd
import json
import os

def ingest_train_data(directory: str):
    all_data = []
    for file in os.listdir(directory):
        path = os.path.join(directory, file)
        with open(path, 'r') as f:
            data = json.load(f)
            company_name = data['company']
            # drop keys that don't matter
            del_keys = [
                'state_location'
                'cik',
                'company',
                'filing_type',
                'filing_date',
                'period_of_report',
                'sic',
                'state_of_inc',
                'fiscal_year_end',
                'filing_html_index',
                'htm_filing_link',
                'complete_text_filing_link',
                'filename'
            ]
            all_data.extend([(data[text], text, company_name) for text in data.keys() if text not in del_keys])
    return all_data


def create_master_dataset(directory: str):
    df = pd.DataFrame(columns=["prompt", "completion"])
    data = ingest_train_data(directory=directory)
    for completion, item, company in data:
        prompt = f"What did {item} in {company}'s SEC 10-K filing say?"
        df.loc[len(df.index)] = [prompt, completion]
    return df

directory = './10-K/'
master_set = create_master_dataset(directory=directory)

In [6]:
print("sanity check")

sanity check


In [7]:
!pip install datasets


Defaulting to user installation because normal site-packages is not writeable


In [8]:

# format dataset how unsloth wants
from unsloth import to_sharegpt
from datasets import Dataset

dataset = Dataset.from_pandas(master_set)

dataset = to_sharegpt(
    dataset,
    merged_prompt = "Your input is: {prompt}",
    output_column_name='completion',
)

from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Standardizing format: 100%|██████████| 11960/11960 [00:00<00:00, 24403.82 examples/s]


In [9]:
master_set

,prompt,completion
0,"What did cik in Mondelez International, Inc.'s...",1103982
1,What did state_location in Mondelez Internatio...,IL
2,"What did item_1 in Mondelez International, Inc...",Item 1. Business.\nGeneral\nMondelēz Internati...
3,"What did item_1A in Mondelez International, In...",Item 1A. Risk Factors.\nYou should carefully r...
4,"What did item_1B in Mondelez International, In...",Item 1B. Unresolved Staff Comments.\nNone.
...,...,...
11955,What did item_13 in ADVANCED MICRO DEVICES INC...,ITEM 13. CERTAIN RELATIONSHIPS AND RELATED TRA...
11956,What did item_14 in ADVANCED MICRO DEVICES INC...,ITEM 14. PRINCIPAL ACCOUNTING FEES AND SERVICE...
11957,What did item_15 in ADVANCED MICRO DEVICES INC...,"ITEM 15. EXHIBITS, FINANCIAL STATEMENT SCHEDUL..."
11958,What did item_16 in ADVANCED MICRO DEVICES INC...,ITEM 16. FORM 10-K SUMMARY\nNot applicable.


In [10]:

# Now it's time to start training the model
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=4,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=50,
        max_steps=1000,
        learning_rate=5e-6,
        fp16=not is_bfloat16_supported(),
#         fp16=True,
        bf16=is_bfloat16_supported(),
        logging_steps=10,
        output_dir="outputs1",
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="cosine",
        report_to='none'
    ),
)

Map (num_proc=4): 100%|██████████| 11960/11960 [00:39<00:00, 303.94 examples/s]
max_steps is given, it will override any value given in num_train_epochs


In [11]:
trainer_stats = trainer.train()


==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 11,960 | Num Epochs = 2
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 4
\        /    Total batch size = 16 | Total steps = 1,000
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss
10,1.692500
20,1.751700
30,1.640900
40,1.659200
50,1.630500
60,1.475400
70,1.358800
80,1.433900
90,1.303200
100,1.252100


In [12]:
model.save_pretrained('mistral-7b-ft-3')
tokenizer.save_pretrained('mistral-7b-ft-3')

('mistral-7b-ft-3/tokenizer_config.json',
 'mistral-7b-ft-3/special_tokens_map.json',
 'mistral-7b-ft-3/tokenizer.model',
 'mistral-7b-ft-3/added_tokens.json',
 'mistral-7b-ft-3/tokenizer.json')

In [13]:
%%capture
!curl -fsSL https://ollama.com/install.sh | sh

In [20]:
!pwd

/storage/ice1/5/1/mmahendran7


In [25]:
!./bin/bin/ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model from a Modelfile
  show        Show information for a model
  run         Run a model
  stop        Stop a running model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


In [17]:

# Save this in 8B for now (faster)
model.save_pretrained_gguf("ft_mistral", tokenizer)

Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 1658.54 out of 2015.05 RAM for saving.


100%|██████████| 32/32 [00:00<00:00, 119.28it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Done.
==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp will take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits will take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q8_0'] will take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: [0] Installing llama.cpp. This will take 3 minutes...
Unsloth: [1] Converting model at ft_mistral into q8_0 GGUF format.
The output location will be /storage/ice1/5/1/mmahendran7/ft_mistral/unsloth.Q8_0.gguf
This will take 3 minutes...
INFO:hf-to-gguf:Loading model: ft_mistral
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'model.safetensors.index.json'
INFO:hf-to-gguf:gguf: loading model part 'model-00001-of-00003.safetensors'
I

In [26]:
import subprocess
import os

ollama_path = os.path.abspath("./bin/bin/ollama")
subprocess.Popen([ollama_path, "serve"])
import time
time.sleep(5)

2024/11/29 15:05:24 routes.go:1197: INFO server config env="map[CUDA_VISIBLE_DEVICES:0 GPU_DEVICE_ORDINAL: HIP_VISIBLE_DEVICES: HSA_OVERRIDE_GFX_VERSION: HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_INTEL_GPU:false OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/home/hice1/mmahendran7/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://*] OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: ROCR_VISIBLE_DEVICES: http_proxy: https_proxy: no_proxy:]"
time=2024-11-29T15:05:24.675-05:00 level=I

In [22]:
ls -l /usr/local/bin/ollama

ls: cannot access '/usr/local/bin/ollama': No such file or directory


In [27]:

# # Try to create the model file from the GGUF thing
# with open('ft_model/Modelfile', 'w') as f:
#     f.write("FROM ft_model/unsloth.Q8_0.gguf")
    
    
import os

gguf_path = os.path.abspath("/storage/ice1/5/1/mmahendran7/ft_mistral/unsloth.Q8_0.gguf")

with open('ft_mistral/Modelfile', 'w') as f:
    f.write(f"FROM {gguf_path}")


In [9]:
f

<_io.TextIOWrapper name='ft_mistral/Modelfile' mode='w' encoding='UTF-8'>

In [28]:
!./bin/bin/ollama create ft_mistral -f ./ft_mistral/Modelfile


[GIN] 2024/11/29 - 15:05:56 | 200 |      54.236µs |       127.0.0.1 | HEAD     "/"
transferring model data ⠋ transferring model data ⠹ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠴ transferring model data ⠦ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠋ transferring model data ⠙ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠦ transferring model data ⠇ transferring model data ⠏ transferring model data ⠋ transferring model data ⠋ transferring model data ⠙ transferring model data ⠸ transferring model data ⠼ transferring model data ⠴ transferring model data ⠦ transferring model data ⠦ transferring model data ⠧ transferring model data ⠏ transferring model data ⠋ transferring model data ⠙ transferring model data ⠙ transferring model data ⠹ transferring model data ⠼ transferring model data ⠴ transfe

In [11]:
# Start running inference using eval dataset
# %%capture
!pip install -U langchain-ollama

Defaulting to user installation because normal site-packages is not writeable


In [29]:
prompt_template = """
You are an expert financial analyst tasked with providing clear and accurate insights based on a company's financial history, derived from SEC filings you have been trained on. Your goal is to generate precise, data-driven responses to financial queries in a professional tone.

Your response must:
1. Be concise and well-structured, limited to 500 words.
2. Focus only on the financial insights directly relevant to the question.
3. Avoid adding any unnecessary context, disclaimers, or formatting such as markdown.
4. Exclude speculation or personal opinions—only rely on the provided data.
5. Do not repeat yourself in your responses. Do not give the same line of text more than once.

Here is an example, the question and company will be given to you, and you have to generate a response:

### Question: What do AAPL's earnings reports say about its growth potential?

### Company: Apple Inc.

### Response:
Apple Inc. has demonstrated robust growth potential. Its earnings reports highlight consistent revenue growth exceeding 2% annually, driven by strong product demand and effective cost management.

Here is your actual question and company it relates to:
### Question: 
{}

### Company: 
{}

### Response: 
"""

In [13]:
# from langchain_core.prompts import ChatPromptTemplate
# from langchain_ollama.llms import OllamaLLM
# from tqdm import tqdm
# import pandas as pd

# prompt = ChatPromptTemplate.from_template(prompt_template)

# model = OllamaLLM(model="ft_model_mistral")

# chain = prompt | model

# eval_dataset = pd.read_csv('evaluation_dataset.csv')

# idno = 0
# answers = []

# for question in tqdm(eval_dataset["Question"]):
#   output = chain.invoke({'query': question})
#   answers.append({
#       "no": idno,
#       "question":question,
#       "answer": output
#   })
#   idno += 1

# final = pd.DataFrame(answers)
# final.to_csv("mistral-7b-ft-output.csv", index=False)

In [ ]:
import pandas as pd
from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

EOS_TOKEN = tokenizer.eos_token
prompt = ChatPromptTemplate.from_template(prompt_template) + EOS_TOKEN
model = OllamaLLM(model="ft_model_mistral", max_tokens=500, stop=[EOS_TOKEN])
chain = prompt | model

eval_dataset = pd.read_csv('evaluation_dataset.csv')
batch_size = 10
checkpoint_interval = 200

results = []



for i in tqdm(range(0, len(eval_dataset), batch_size)):
    batch = eval_dataset.iloc[i:i+batch_size]
    questions = batch['Question'].tolist()        
    companies = batch['Company'].tolist()  # Access company names

    
#     row = eval_dataset.iloc[i]
#     question = row['Question']
    
#     output = chain.invoke({'query': question
    print(f"Processing batch {i//batch_size + 1}")
    print(f"Questions: {questions}")
    
    
    queries = [
        prompt_template.format(query=c+ " " + q).strip()
        for q,c in zip(questions,companies)
    ]

#     queries = [
#         prompt_template.format(query=q, company=c).strip()
#         for q, c in zip(questions, companies)
#     ]
    
    outputs = chain.batch([
        {'query': query} for query in queries
    ])

    
#     results.append({
#         "question_id": row.name,
#         "company": row['Company'],
#         "question": question,
#         "ground_truth": row['Answer'],
#         "model_output": output
#     })
    
#     if (i + 1) % checkpoint_interval == 0:
#         pd.DataFrame(results).to_csv(f"mistral-7b-ft-output_partial_{i+1}.csv", index=False)
    for j, output in enumerate(outputs):
        print(f"Question ID: {batch.index[j]}")
        print(f"Question: {questions[j]}")
        print(f"Model Output: {output.strip()}")
        results.append({
            "question_id": batch.index[j],
            "company": batch.iloc[j]['Company'],
            "question": questions[j],
            "ground_truth": batch.iloc[j]['Answer'],
            "model_output": output.strip()
        })

    if (i + batch_size) % checkpoint_interval == 0 or (i + batch_size) >= len(eval_dataset):
        pd.DataFrame(results).to_csv(f"model_mistral-7b-ft-output_partial_{i+batch_size}.csv", index=False)
    

final = pd.DataFrame(results)
final.to_csv("model_mistral-7b-ft-output_final.csv", index=False)

  0%|          | 0/121 [00:00<?, ?it/s]

Processing batch 1
Questions: ["What is Meta's current dividend policy and how does it affect investors looking for dividends?", "DEXCOM INC's 2020 annual report mentions a number of risks and uncertainties that could affect the company's business. What are some of the most significant risks that DEXCOM INC faces in the near future?", "DEXCOM INC is a medical device company that focuses on continuous glucose monitoring systems. Based on the information provided, what are some of the biggest risks that could affect the company's future financial performance?", 'DEXCOM INC is a medical device company that focuses on continuous glucose monitoring systems. What are some of the risks associated with their business?', "Given the information provided, what is the company's outlook on its financial performance in the coming year?", "Given T-Mobile's financial history, what are some of the main risks associated with the company's debt levels?", 'ON Semiconductor has experienced a decrease in re

time=2024-11-28T19:47:27.686-05:00 level=INFO source=sched.go:714 msg="new model will fit in available VRAM in single GPU, loading" model=/home/hice1/mmahendran7/.ollama/models/blobs/sha256-3dd6bd2319a494b752856015b58dd5ea62b9bc197ca0420fdc00c8c9936b7521 gpu=GPU-14697b4a-1ecb-a9db-7a0f-715e67b5e23c parallel=4 available=78165377024 required="9.3 GiB"
time=2024-11-28T19:47:27.973-05:00 level=INFO source=server.go:105 msg="system memory" total="2015.0 GiB" free="1940.8 GiB" free_swap="8.0 GiB"
time=2024-11-28T19:47:27.973-05:00 level=INFO source=memory.go:343 msg="offload to cuda" layers.requested=-1 layers.model=33 layers.offload=33 layers.split="" memory.available="[72.8 GiB]" memory.gpu_overhead="0 B" memory.required.full="9.3 GiB" memory.required.partial="9.3 GiB" memory.required.kv="1.0 GiB" memory.required.allocations="[9.3 GiB]" memory.weights.total="7.9 GiB" memory.weights.repeating="7.8 GiB" memory.weights.nonrepeating="136.0 MiB" memory.graph.full="560.0 MiB" memory.graph.partia

[GIN] 2024/11/28 - 19:47:30 | 200 |  3.431993993s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/28 - 19:47:32 | 200 |  1.521895196s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/28 - 19:47:34 | 200 |  2.406714643s |       127.0.0.1 | POST     "/api/generate"


In [1]:
results


NameError: name 'results' is not defined

In [32]:
import pandas as pd
from tqdm import tqdm
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM

EOS_TOKEN = tokenizer.eos_token
prompt = ChatPromptTemplate.from_template(prompt_template) + EOS_TOKEN
model = OllamaLLM(model="ft_model_mistral", max_tokens=500, stop=[EOS_TOKEN])
chain = prompt | model

eval_dataset = pd.read_csv('evaluation_dataset.csv')
batch_size = 10
checkpoint_interval = 200

results = []

for i in tqdm(range(0, len(eval_dataset), batch_size)):
    batch = eval_dataset.iloc[i:i+batch_size]
    questions = batch['Question'].tolist()
    companies = batch['Company'].tolist()


    prompts = [prompt_template.format(q,c).strip() + EOS_TOKEN for q,c in zip(questions, companies)]
#     print(prompts)
    outputs = model.batch(prompts)
    
    

    for j, output in enumerate(outputs):
        results.append({
            "question_id": batch.index[j],
            "company": batch.iloc[j]['Company'],
            "question": questions[j],
            "ground_truth": batch.iloc[j]['Answer'],
            "model_output": output.strip(),
            
        })
    
#     if i==20:
#         break
    
    if (i + batch_size) % checkpoint_interval == 0 or (i + batch_size) >= len(eval_dataset):
        pd.DataFrame(results).to_csv(f"model_mistral-7b-ft-output_partial_{i+batch_size}.csv", index=False)
        

final = pd.DataFrame(results)
final.to_csv("finetune_output.csv", index=False)
        


  0%|          | 0/121 [00:00<?, ?it/s]

[GIN] 2024/11/29 - 15:10:03 | 200 |  1.209957183s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:07 | 200 |  4.181728544s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:12 | 200 |  4.382737858s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:17 | 200 |  5.211523156s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:19 | 200 |  2.040172362s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:22 | 200 |  3.463649052s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:24 | 200 |  2.215485752s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:27 | 200 |  2.954580152s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:29 | 200 |  1.074845904s |       127.0.0.1 | POST     "/api/generate"


  1%|          | 1/121 [00:32<1:04:32, 32.27s/it]

[GIN] 2024/11/29 - 15:10:34 | 200 |  5.520092003s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:35 | 200 |  1.122823649s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:40 | 200 |  5.180584542s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:41 | 200 |  941.925978ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:45 | 200 |  3.279134692s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:47 | 200 |  2.692767022s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:48 | 200 |  938.950523ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:51 | 200 |  2.453070408s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:52 | 200 |  1.715144435s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 15:10:56 | 200 |  3.751133011s |       127.0.0.1 | POST     "/api/generate"


  2%|▏         | 2/121 [00:59<58:11, 29.34s/it]  

...

[GIN] 2024/11/29 - 16:47:04 | 200 |  1.999200947s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:05 | 200 |  1.212401305s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:08 | 200 |  2.434343233s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:09 | 200 |   962.51289ms |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:11 | 200 |  2.354945116s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:12 | 200 |   1.57063499s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:14 | 200 |  1.314541118s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:15 | 200 |  1.412190806s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:20 | 200 |   5.18037508s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:24 | 200 |  3.684069106s |       127.0.0.1 | POST     "/api/generate"


 99%|█████████▉| 120/121 [1:37:25<00:28, 28.58s/it]

[GIN] 2024/11/29 - 16:47:28 | 200 |  3.504965972s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:30 | 200 |   1.85547277s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:31 | 200 |  1.283733953s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:35 | 200 |  3.904293761s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:38 | 200 |  3.109726202s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:40 | 200 |  1.845732539s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:42 | 200 |  2.410371718s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:46 | 200 |  3.665022436s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:47 | 200 |  1.451462964s |       127.0.0.1 | POST     "/api/generate"
[GIN] 2024/11/29 - 16:47:50 | 200 |  3.213823281s |       127.0.0.1 | POST     "/api/generate"


100%|██████████| 121/121 [1:37:50<00:00, 48.51s/it]

[GIN] 2024/11/29 - 16:47:52 | 200 |  1.407894813s |       127.0.0.1 | POST     "/api/generate"


In [31]:
final = pd.DataFrame(results)
final.to_csv("newtest_mistral-7b-ft-output_final.csv", index=False)

In [45]:
print(eval_dataset['Question'].head())  # Verify sample questions


0    What is Meta's current dividend policy and how...
1    DEXCOM INC's 2020 annual report mentions a num...
2    DEXCOM INC is a medical device company that fo...
3    DEXCOM INC is a medical device company that fo...
4    Given the information provided, what is the co...
Name: Question, dtype: object


In [33]:
!pip install sacrebleu
!pip install bert_score

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 2.1 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 1.3 MB/s eta 0:00:00a 0:00:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [34]:
import pandas as pd

# Load the finetune output CSV file
finetune_output = pd.read_csv("finetune_output.csv")

# Extract the ground_truth column as references
references = finetune_output['ground_truth'].tolist()

# Extract the model_output column as hypothesis
hypothesis = finetune_output['model_output'].tolist()





In [35]:
from sacrebleu import corpus_bleu

references_for_bleu = [[ref] for ref in references]
bleu = corpus_bleu(hypothesis, references_for_bleu)
print(f"BLEU Score: {bleu.score:.4f}")

BLEU Score: 39.7422


In [36]:
from bert_score import score as bert_score

P, R, F1 = bert_score(hypothesis, references, lang="en", rescale_with_baseline=True)
print(f"BERTScore - Precision: {P.mean().item():.4f}, Recall: {R.mean().item():.4f}, F1: {F1.mean().item():.4f}")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BERTScore - Precision: 0.1818, Recall: 0.2105, F1: 0.1962
